In [168]:
from seances.models import Seance
print("Number of completed seances:")
print(Seance.objects.filter(valid=True).count())
    
print("Experiment 1 count: {}".format(Seance.objects.filter(experiment__sequence_number=1).count()))
print("Experiment 2 count: {}".format(Seance.objects.filter(experiment__sequence_number=2).count()))
print("Experiment 3 count: {}".format(Seance.objects.filter(experiment__sequence_number=3).count()))


Number of completed seances:


InterfaceError: connection already closed

In [ ]:
# Loading the data from the db
# should we use Influx and interpolated data (probably not)?
# it will take some time to calculate all features for everyone.

from numpy import mean, std
from sklearn.preprocessing import normalize

def load_data(username, seance_num, sens):
    try:
        seance = Seance.objects.filter(user__username=username, valid=True).order_by('created')[seance_num]
    except IndexError:
        print("Invalid seance id.")
        return
    if sens == "accelerometer":
        sensor_ids = [60,61,62]
        sensors = Sensor.objects.filter(id__in=sensor_ids).order_by('id')
        return (
                SensorRecord.objects.filter(seance=seance, sensor=sensors[0]).order_by("created"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[1]).order_by("created"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[2]).order_by("created")
        )
    elif sens == "gyroscope":
        sensor_ids = [63,64,65]
        sensors = Sensor.objects.filter(id__in=sensor_ids).order_by('id')
        return (
                SensorRecord.objects.filter(seance=seance, sensor=sensors[0]).order_by("created"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[1]).order_by("created"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[2]).order_by("created")
        )
    elif sens == "force":
        sensor_ids = [54,55,76,77]
        sensors = Sensor.objects.filter(id__in=sensor_ids).order_by('topic')
        return (
                SensorRecord.objects.filter(seance=seance, sensor=sensors[0]).order_by("created"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[1]).order_by("created"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[2]).order_by("created"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[3]).order_by("created")
        )
    elif sens == "cpu":
        sensor_ids = [78,79,80,81]
        sensors = Sensor.objects.filter(id__in=sensor_ids).order_by('topic')
        return (
                SensorRecord.objects.filter(seance=seance, sensor=sensors[0]).order_by("created"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[1]).order_by("created"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[2]).order_by("created"), 
                SensorRecord.objects.filter(seance=seance, sensor=sensors[3]).order_by("created")
        )
    elif sens == "ram":
        sensor_ids = [82]
        sensors = Sensor.objects.filter(id__in=sensor_ids).order_by('topic')
        return SensorRecord.objects.filter(seance=seance, sensor=sensors[0]).order_by("created")
    elif sens == "net":
        sensor_ids = [83, 84]
        sensors = Sensor.objects.filter(id__in=sensor_ids).order_by('id')
        return (
            SensorRecord.objects.filter(seance=seance, sensor=sensors[0]).order_by("created"),
            SensorRecord.objects.filter(seance=seance, sensor=sensors[1]).order_by("created")
        )
    elif sens == "pir":
        sensor_ids = [58, 59, 66, 67, 68, 69]
        sensors = Sensor.objects.filter(id__in=sensor_ids).order_by('id')
        return (
            SensorRecord.objects.filter(seance=seance, sensor__in=sensors).order_by("created"),
            seance.start,
            seance.end,
        )
    else:
        raise ValueError("Invalid sensor string.")
    

In [159]:
# Functions
from scipy.signal import find_peaks
from datetime import datetime, timedelta

def process_signal(records):
    """
    Take Django query and do basic signal processing.
    """
    values = [x.value for x in records]
    times = [x.timestamp for x in records]
    m = mean(values)
    s = std(values)
    norm = [(x - m)/s for x in values]
    
    return values, times, norm, m, s

def process_binary_signal(records):
    """
    Take Django query with hall or pir data and return meaningful statistics.
    """
    if len(records) == 0:
        return [], []
    values = [x.value for x in records]
    times = [x.timestamp for x in records]
    ids = [x.sensor_id for x in records]
    return values, times, ids
    
def mean_crossing_rate(signal, m):
    """
    Calculate mean crossing rate from signal.
    Rate of mean crossings vs. the signal length.
    """
    prev = signal[0]
    crosses = 0
    length = len(signal) - 1
    
    for curr in signal[1:]:
        if prev <= m < curr or prev > m >=curr:
            crosses += 1
        prev = curr
    return crosses/length

def mean_acceleration_intensity(signal):
    """
    Mean derivative of a signal.
    """
    prev = signal[0]
    length = len(signal) - 1
    derv = []
    
    for curr in signal[1:]:
        derv.append(abs(curr - prev))
        prev = curr
        
    return mean(derv)
    
    
def find_typing_start_and_stop(signal):
    """
    Find the two peaks that are present in the signal.
    (specific for experiment 1)
    """
    peaks_id = [0,1]
    for i in range(2, len(signal)):
        x = abs(signal[i])
        if x > signal[peaks_id[0]]:
            peaks_id[0] = i
            continue
        if x > signal[peaks_id[1]]:
            peaks_id[1] = i
    print(signal[peaks_id[0]])
    print(signal[peaks_id[1]])
    

def create_time_chunks(start: datetime, end: datetime, interval: timedelta):
    """
    Create a list of time intervals for given parameters.
    """
    intervals = []
    while start < end:
        intervals.append((start, min(start + interval, end)))
        start += interval
    return intervals
    

# Accelerometer data overview

In [142]:
x_val_1, y_val_1, z_val_1 = load_data("test_subject_01", 0, "accelerometer")
x_val_2, y_val_2, z_val_2 = load_data("test_subject_02", 0, "accelerometer")
x_val_3, y_val_3, z_val_3 = load_data("test_subject_03", 0, "accelerometer")
x_val_4, y_val_4, z_val_4 = load_data("test_subject_04", 0, "accelerometer")

x_val_1, x_tim_1, x_nor_1, x_m_1, x_s_1 = process_signal(x_val_1)
y_val_1, y_tim_1, y_nor_1, y_m_1, y_s_1 = process_signal(y_val_1)
z_val_1, z_tim_1, z_nor_1, z_m_1, z_s_1 = process_signal(z_val_1)

x_val_2, x_tim_2, x_nor_2, x_m_2, x_s_2 = process_signal(x_val_2)
y_val_2, y_tim_2, y_nor_2, y_m_2, y_s_2 = process_signal(y_val_2)
z_val_2, z_tim_2, z_nor_2, z_m_2, z_s_2 = process_signal(z_val_2)

x_val_3, x_tim_3, x_nor_3, x_m_3, x_s_3 = process_signal(x_val_3)
y_val_3, y_tim_3, y_nor_3, y_m_3, y_s_3 = process_signal(y_val_3)
z_val_3, z_tim_3, z_nor_3, z_m_3, z_s_3 = process_signal(z_val_3)

x_val_4, x_tim_4, x_nor_4, x_m_4, x_s_4 = process_signal(x_val_4)
y_val_4, y_tim_4, y_nor_4, y_m_4, y_s_4 = process_signal(y_val_4)
z_val_4, z_tim_4, z_nor_4, z_m_4, z_s_4 = process_signal(z_val_4)

## Mean crossing rate on a single signal
- how to merge signals into one?
- interpolation to achieve a static sampling interval?
- mean crossing rate for time chunks as a vector?
- use time warping on values?

In [143]:
print("X mean crossing rate:")
print(mean_crossing_rate(x_val_1, x_m_1))
print(mean_crossing_rate(x_val_2, x_m_2))
print(mean_crossing_rate(x_val_3, x_m_3))
print(mean_crossing_rate(x_val_4, x_m_4))
print()
print("Y mean crossing rate:")
print(mean_crossing_rate(y_val_1, y_m_1))
print(mean_crossing_rate(y_val_2, y_m_2))
print(mean_crossing_rate(y_val_3, y_m_3))
print(mean_crossing_rate(y_val_4, y_m_4))
print()
print("Z mean crossing rate:")
print(mean_crossing_rate(z_val_1, z_m_1))
print(mean_crossing_rate(z_val_2, z_m_2))
print(mean_crossing_rate(z_val_3, z_m_3))
print(mean_crossing_rate(z_val_4, z_m_4))

X mean crossing rate:
0.41755978026712626
0.22630066172606472
0.39876727479678287
0.22855036282968347

Y mean crossing rate:
0.39735028005170187
0.35534260679314894
0.4726448827944635
0.23297718218036942

Z mean crossing rate:
0.411851034037053
0.4376454563979365
0.45526002126128834
0.44841413566513166


## Mean acceleration intensity on a single signal
- Calculated by taking the mean derivative of a signal.

In [144]:
print("X mean acceleration intensity:")
print(mean_acceleration_intensity(x_val_1))
print(mean_acceleration_intensity(x_val_2))
print(mean_acceleration_intensity(x_val_3))
print(mean_acceleration_intensity(x_val_4))
print()
print("Y mean acceleration intensity:")
print(mean_acceleration_intensity(y_val_1))
print(mean_acceleration_intensity(y_val_2))
print(mean_acceleration_intensity(y_val_3))
print(mean_acceleration_intensity(y_val_4))
print()
print("Z mean acceleration intensity:")
print(mean_acceleration_intensity(z_val_1))
print(mean_acceleration_intensity(z_val_2))
print(mean_acceleration_intensity(z_val_3))
print(mean_acceleration_intensity(z_val_4))

X mean acceleration intensity:
0.010863245906936667
0.008716283609141902
0.008438798629828085
0.008669883196005639

Y mean acceleration intensity:
0.008125740521327016
0.007799734022586631
0.008014431904818153
0.009671702799192841

Z mean acceleration intensity:
0.03960334984920293
0.0294240731008891
0.03078752671083574
0.03911613287111295


In [145]:
find_peaks(x_val, threshold=0.2)
find_peaks(y_val, threshold=0.2)
find_peaks(z_val, threshold=0.2)


# find_typing_start_and_stop(x_val)
# print()
# find_typing_start_and_stop(y_val)
# print()
# find_typing_start_and_stop(z_val)

NameError: name 'x_val' is not defined

## Dynamic type warping example

In [146]:
import numpy as np
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

x = np.array([[1,1], [2,2], [3,3], [4,4], [5,5]])
y = np.array([[2,2], [3,3], [3,3], [4,4]])

distance, path = fastdtw(x, y, dist=euclidean)
print(distance)
print(path)


2.8284271247461903
[(0, 0), (1, 0), (2, 1), (2, 2), (3, 3), (4, 3)]


# Gyroscope data overview

In [147]:
x_val_1_g, y_val_1_g, z_val_1_g = load_data("test_subject_01", 0, "gyroscope")
x_val_2_g, y_val_2_g, z_val_2_g = load_data("test_subject_02", 0, "gyroscope")
x_val_3_g, y_val_3_g, z_val_3_g = load_data("test_subject_03", 0, "gyroscope")
x_val_4_g, y_val_4_g, z_val_4_g = load_data("test_subject_04", 0, "gyroscope")

x_val_1_g, x_tim_1_g, x_nor_1_g, x_m_1_g, x_s_1_g = process_signal(x_val_1_g)
y_val_1_g, y_tim_1_g, y_nor_1_g, y_m_1_g, y_s_1_g = process_signal(y_val_1_g)
z_val_1_g, z_tim_1_g, z_nor_1_g, z_m_1_g, z_s_1_g = process_signal(z_val_1_g)

x_val_2_g, x_tim_2_g, x_nor_2_g, x_m_2_g, x_s_2_g = process_signal(x_val_2_g)
y_val_2_g, y_tim_2_g, y_nor_2_g, y_m_2_g, y_s_2_g = process_signal(y_val_2_g)
z_val_2_g, z_tim_2_g, z_nor_2_g, z_m_2_g, z_s_2_g = process_signal(z_val_2_g)

x_val_3_g, x_tim_3_g, x_nor_3_g, x_m_3_g, x_s_3_g = process_signal(x_val_3_g)
y_val_3_g, y_tim_3_g, y_nor_3_g, y_m_3_g, y_s_3_g = process_signal(y_val_3_g)
z_val_3_g, z_tim_3_g, z_nor_3_g, z_m_3_g, z_s_3_g = process_signal(z_val_3_g)

x_val_4_g, x_tim_4_g, x_nor_4_g, x_m_4_g, x_s_4_g = process_signal(x_val_4_g)
y_val_4_g, y_tim_4_g, y_nor_4_g, y_m_4_g, y_s_4_g = process_signal(y_val_4_g)
z_val_4_g, z_tim_4_g, z_nor_4_g, z_m_4_g, z_s_4_g = process_signal(z_val_4_g)

## Mean crossing rate

In [148]:
print("X mean crossing rate:")
print(mean_crossing_rate(x_val_1_g, x_m_1_g))
print(mean_crossing_rate(x_val_2_g, x_m_2_g))
print(mean_crossing_rate(x_val_3_g, x_m_3_g))
print(mean_crossing_rate(x_val_4_g, x_m_4_g))
print()
print("Y mean crossing rate:")
print(mean_crossing_rate(y_val_1_g, y_m_1_g))
print(mean_crossing_rate(y_val_2_g, y_m_2_g))
print(mean_crossing_rate(y_val_3_g, y_m_3_g))
print(mean_crossing_rate(y_val_4_g, y_m_4_g))
print()
print("Z mean crossing rate:")
print(mean_crossing_rate(z_val_1_g, z_m_1_g))
print(mean_crossing_rate(z_val_2_g, z_m_2_g))
print(mean_crossing_rate(z_val_3_g, z_m_3_g))
print(mean_crossing_rate(z_val_4_g, z_m_4_g))

X mean crossing rate:
0.4932477919000431
0.46970753209425037
0.44458642498953044
0.5183292802814715

Y mean crossing rate:
0.45126696467040067
0.4208931693139284
0.45349898525669247
0.4554121177627154

Z mean crossing rate:
0.4975360835846618
0.4946697268369065
0.5089878229495522
0.4985771200910643


# Force sensors data overview

In [149]:
f1_1, f2_1, f3_1, f4_1 = load_data("test_subject_01", 0, "force")
f1_2, f2_2, f3_2, f4_2 = load_data("test_subject_02", 0, "force")
f1_3, f2_3, f3_3, f4_3 = load_data("test_subject_03", 0, "force")
f1_4, f2_4, f3_4, f4_4 = load_data("test_subject_04", 0, "force")

f1_val_1, f1_tim_1, f1_nor_1, f1_m_1, f1_s_1 = process_signal(f1_1)
f2_val_1, f2_tim_1, f2_nor_1, f2_m_1, f2_s_1 = process_signal(f2_1)
f3_val_1, f3_tim_1, f3_nor_1, f3_m_1, f3_s_1 = process_signal(f3_1)
f4_val_1, f4_tim_1, f4_nor_1, f4_m_1, f4_s_1 = process_signal(f4_1)

f1_val_2, f1_tim_2, f1_nor_2, f1_m_2, f1_s_2 = process_signal(f1_2)
f2_val_2, f2_tim_2, f2_nor_2, f2_m_2, f2_s_2 = process_signal(f2_2)
f3_val_2, f3_tim_2, f3_nor_2, f3_m_2, f3_s_2 = process_signal(f3_2)
f4_val_2, f4_tim_2, f4_nor_2, f4_m_2, f4_s_2 = process_signal(f4_2)

f1_val_3, f1_tim_3, f1_nor_3, f1_m_3, f1_s_3 = process_signal(f1_3)
f2_val_3, f2_tim_3, f2_nor_3, f2_m_3, f2_s_3 = process_signal(f2_3)
f3_val_3, f3_tim_3, f3_nor_3, f3_m_3, f3_s_3 = process_signal(f3_3)
f4_val_3, f4_tim_3, f4_nor_3, f4_m_3, f4_s_3 = process_signal(f4_3)

f1_val_4, f1_tim_4, f1_nor_4, f1_m_4, f1_s_4 = process_signal(f1_4)
f2_val_4, f2_tim_4, f2_nor_4, f2_m_4, f2_s_4 = process_signal(f2_4)
f3_val_4, f3_tim_4, f3_nor_4, f3_m_4, f3_s_4 = process_signal(f3_4)
f4_val_4, f4_tim_4, f4_nor_4, f4_m_4, f4_s_4 = process_signal(f4_4)

## Mean crossing rate

In [150]:
print("Fsr 1 mean crossing rate:")
print(mean_crossing_rate(f1_val_1, f1_m_1) * 100)
print(mean_crossing_rate(f1_val_2, f1_m_2) * 100)
print(mean_crossing_rate(f1_val_3, f1_m_3) * 100)
print(mean_crossing_rate(f1_val_4, f1_m_4) * 100)
print()
print("Fsr 2 mean crossing rate:")
print(mean_crossing_rate(f2_val_1, f2_m_1) * 100)
print(mean_crossing_rate(f2_val_2, f2_m_2) * 100)
print(mean_crossing_rate(f2_val_3, f2_m_3) * 100)
print(mean_crossing_rate(f2_val_4, f2_m_4) * 100)
print()
print("Fsr 3 mean crossing rate:")
print(mean_crossing_rate(f3_val_1, f3_m_1) * 100)
print(mean_crossing_rate(f3_val_2, f3_m_2) * 100)
print(mean_crossing_rate(f3_val_3, f3_m_3) * 100)
print(mean_crossing_rate(f3_val_4, f3_m_4) * 100)
print()
print("Fsr 4 mean crossing rate:")
print(mean_crossing_rate(f4_val_1, f4_m_1) * 100)
print(mean_crossing_rate(f4_val_2, f4_m_2) * 100)
print(mean_crossing_rate(f4_val_3, f4_m_3) * 100)
print(mean_crossing_rate(f4_val_4, f4_m_4) * 100)


Fsr 1 mean crossing rate:
8.051551010726616
0.16371748534126987
0.4930369345212352
0.0007392021052475957

Fsr 2 mean crossing rate:
6.03453927957934
0.004161378248475895
0.1413366094434599
0.0011911284751173262

Fsr 3 mean crossing rate:
1.3532385279597177
0.6420595856270227
2.1004684497981563
0.0110489098408957

Fsr 4 mean crossing rate:
5.552350735506201
0.7726126424698748
39.45514173458454
0.20429544264012572


# CPU overview

In [151]:
c1_1, c2_1, c3_1, c4_1 = load_data("test_subject_01", 0, "cpu")
c1_2, c2_2, c3_2, c4_2 = load_data("test_subject_02", 0, "cpu")
c1_3, c2_3, c3_3, c4_3 = load_data("test_subject_03", 0, "cpu")
c1_4, c2_4, c3_4, c4_4 = load_data("test_subject_04", 0, "cpu")

c1_val_1, c1_tim_1, c1_nor_1, c1_m_1, c1_s_1 = process_signal(c1_1)
c2_val_1, c2_tim_1, c2_nor_1, c2_m_1, c2_s_1 = process_signal(c2_1)
c3_val_1, c3_tim_1, c3_nor_1, c3_m_1, c3_s_1 = process_signal(c3_1)
c4_val_1, c4_tim_1, c4_nor_1, c4_m_1, c4_s_1 = process_signal(c4_1)
c1_val_2, c1_tim_2, c1_nor_2, c1_m_2, c1_s_2 = process_signal(c1_2)
c2_val_2, c2_tim_2, c2_nor_2, c2_m_2, c2_s_2 = process_signal(c2_2)
c3_val_2, c3_tim_2, c3_nor_2, c3_m_2, c3_s_2 = process_signal(c3_2)
c4_val_2, c4_tim_2, c4_nor_2, c4_m_2, c4_s_2 = process_signal(c4_2)
c1_val_3, c1_tim_3, c1_nor_3, c1_m_3, c1_s_3 = process_signal(c1_3)
c2_val_3, c2_tim_3, c2_nor_3, c2_m_3, c2_s_3 = process_signal(c2_3)
c3_val_3, c3_tim_3, c3_nor_3, c3_m_3, c3_s_3 = process_signal(c3_3)
c4_val_3, c4_tim_3, c4_nor_3, c4_m_3, c4_s_3 = process_signal(c4_3)
c1_val_4, c1_tim_4, c1_nor_4, c1_m_4, c1_s_4 = process_signal(c1_4)
c2_val_4, c2_tim_4, c2_nor_4, c2_m_4, c2_s_4 = process_signal(c2_4)
c3_val_4, c3_tim_4, c3_nor_4, c3_m_4, c3_s_4 = process_signal(c3_4)
c4_val_4, c4_tim_4, c4_nor_4, c4_m_4, c4_s_4 = process_signal(c4_4)

## Mean crossing rate

In [152]:
print("Cpu 1 mean crossing rate:")
print(mean_crossing_rate(c1_val_1, c1_m_1))
print(mean_crossing_rate(c1_val_2, c1_m_2))
print(mean_crossing_rate(c1_val_3, c1_m_3))
print(mean_crossing_rate(c1_val_4, c1_m_4))
print()
print("Cpu 2 mean crossing rate:")
print(mean_crossing_rate(c2_val_1, c2_m_1))
print(mean_crossing_rate(c2_val_2, c2_m_2))
print(mean_crossing_rate(c2_val_3, c2_m_3))
print(mean_crossing_rate(c2_val_4, c2_m_4))
print()
print("Cpu 3 mean crossing rate:")
print(mean_crossing_rate(c3_val_1, c3_m_1))
print(mean_crossing_rate(c3_val_2, c3_m_2))
print(mean_crossing_rate(c3_val_3, c3_m_3))
print(mean_crossing_rate(c3_val_4, c3_m_4))
print()
print("Cpu 4 mean crossing rate:")
print(mean_crossing_rate(c4_val_1, c4_m_1))
print(mean_crossing_rate(c4_val_2, c4_m_2))
print(mean_crossing_rate(c4_val_3, c4_m_3))
print(mean_crossing_rate(c4_val_4, c4_m_4))

Cpu 1 mean crossing rate:
0.11610486891385768
0.2687806719516799
0.2
0.13801452784503632

Cpu 2 mean crossing rate:
0.12195121951219512
0.2612306530766327
0.18333333333333332
0.13317191283292978

Cpu 3 mean crossing rate:
0.13320825515947468
0.270392749244713
0.21
0.14527845036319612

Cpu 4 mean crossing rate:
0.09568480300187618
0.2348942598187311
0.21
0.14805825242718446


# Memory overview

In [153]:
r_1 = load_data("test_subject_01", 0, "ram")
r_2 = load_data("test_subject_02", 0, "ram")
r_3 = load_data("test_subject_03", 0, "ram")
r_4 = load_data("test_subject_04", 0, "ram")

r_val_1, r_tim_1, r_nor_1, r_m_1, r_s_1 = process_signal(r_1)
r_val_2, r_tim_2, r_nor_2, r_m_2, r_s_2 = process_signal(r_2)
r_val_3, r_tim_3, r_nor_3, r_m_3, r_s_3 = process_signal(r_3)
r_val_4, r_tim_4, r_nor_4, r_m_4, r_s_4 = process_signal(r_4)


## Mean crossing rate

In [154]:
print("Memory mean crossing rate:")
print(mean_crossing_rate(r_val_1, r_m_1))
print(mean_crossing_rate(r_val_2, r_m_2))
print(mean_crossing_rate(r_val_3, r_m_3))
print(mean_crossing_rate(r_val_4, r_m_4))

Memory mean crossing rate:
0.022113022113022112
0.005642519111758282
0.05016357688113413
0.008468052347959968


# Network activity overview

In [155]:
ns_1, nr_1 = load_data("test_subject_01", 0, "net")
ns_2, nr_2 = load_data("test_subject_02", 0, "net")
ns_3, nr_3 = load_data("test_subject_03", 0, "net")
ns_4, nr_4 = load_data("test_subject_04", 0, "net")

ns_val_1, ns_tim_1, ns_nor_1, ns_m_1, ns_s_1 = process_signal(ns_1)
ns_val_2, ns_tim_2, ns_nor_2, ns_m_2, ns_s_2 = process_signal(ns_2)
ns_val_3, ns_tim_3, ns_nor_3, ns_m_3, ns_s_3 = process_signal(ns_3)
ns_val_4, ns_tim_4, ns_nor_4, ns_m_4, ns_s_4 = process_signal(ns_4)

nr_val_1, nr_tim_1, nr_nor_1, nr_m_1, nr_s_1 = process_signal(nr_1)
nr_val_2, nr_tim_2, nr_nor_2, nr_m_2, nr_s_2 = process_signal(nr_2)
nr_val_3, nr_tim_3, nr_nor_3, nr_m_3, nr_s_3 = process_signal(nr_3)
nr_val_4, nr_tim_4, nr_nor_4, nr_m_4, nr_s_4 = process_signal(nr_4)

## Jumps statistics

In [156]:
def get_jumps(values, times):
    """
    Get first derivative and empirically determine a threshold for a pair of values to be classified as a jump.
    """
    pv = values[0]
    pt = values[1]
    for cv, ct in zip(values[1:], times[1:]):
        pass

# Pir sensors overview
- Id 58: pir_01
- Id 59: pir_02
- Id 66: pir_03
- Id 67: pir_04
- Id 68: pir_05
- Id 69: pir_06

In [163]:
pir_10, s_sta_10, s_end_10 = load_data("test_subject_01", 2, "pir")
# pir_13, s_sta_13, s_end_13 = load_data("test_subject_01", 3, "pir")
pir_2, s_sta_2, s_end_2 = load_data("test_subject_02", 2, "pir")
pir_3, s_sta_3, s_end_3 = load_data("test_subject_03", 2, "pir")
pir_4, s_sta_4, s_end_4 = load_data("test_subject_04", 2, "pir")

pir_val_10, pir_tim_10, pir_ids_10 = process_binary_signal(pir_10)
pir_val_13, pir_tim_13, pir_ids_13 = process_binary_signal(pir_13)
pir_val_2, pir_tim_2, pir_ids_2 = process_binary_signal(pir_2)
pir_val_3, pir_tim_3, pir_ids_3 = process_binary_signal(pir_3)
pir_val_4, pir_tim_4, pir_ids_4 = process_binary_signal(pir_4)

In [164]:
def detect_pir_activity_type(start, end, values, times, ids, time_interval: timedelta=timedelta(seconds=30)):
    """
    For every given time interval, check which sensors were active and 
    based on that classify each time interval into one of 3 activities:
    - no activity (no sensors active)
    - seating (senor pir_01 active)
    - moving (more than one sensor active)
    """
    if not (len(values) == len(times) and len(times) == len(ids)):
        raise ValueError("Mismatching value lengths.")
    intervals = create_time_chunks(start, end, time_interval)
    result = []
    active = []
    
    for interval in intervals:
        prev = []
        interval_ids = [index for index, value in enumerate(times) if interval[0] <= value <= interval[1]]
        for idx in interval_ids:
            if not values[idx]:
                try:
                    active.remove(ids[idx])
                    prev.append(ids[idx])
                except:
                    pass
            elif ids[idx] not in active:
                active.append(ids[idx])
        result.append({"interval": interval, "ids": set(prev + active)})
        prev = active
        
    for res in result:
        if len(res["ids"]) > 1 or len(res["ids"]) == 1 and res["ids"] != 58:
            res.update({"activity": "movement"})
        elif len(res["ids"]) == 1 and res["ids"] == 58:
            res.update({"activity": "seated"})
        else:
            res.update({"activity": "no activity"})
    return result


def get_activity_vector(intervals):
    """
    Create an enum vector from pir activity intervals.
    """
    result_vector = []
    for x in intervals:
        if x["activity"] == "no activity":
            result_vector.append(0)
        elif x["activity"] == "seated":
            result_vector.append(1)
        elif x["activity"] == "movement":
            result_vector.append(2)
        else:
            raise ValueError("Unknown activity class.")
    return result_vector
    
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

# print(pir_ids_10)
# print(pir_ids_13)
# print(pir_ids_2)
# print(pir_ids_3)
# print(pir_ids_4)

# distance, path = fastdtw(pir_ids_10, pir_ids_2, dist=euclidean)
# print(path)
# print(distance)

# distance, path = fastdtw(pir_ids_13, pir_ids_2, dist=euclidean)
# print(path)
# print(distance)


# distance, path = fastdtw(pir_ids_10, pir_ids_13, dist=euclidean)
# print(path)
# print(distance)

# for i in range(0, len(pir_val_10)):
#     print("============================")
#     print(pir_ids_10[i])
#     print(pir_val_10[i])
#     print(pir_tim_10[i])
    
seconds = 30
intervals_10 = detect_pir_activity_type(s_sta_10, s_end_10, pir_val_10, pir_tim_10, pir_ids_10, timedelta(seconds=seconds))
# intervals_13 = detect_pir_activity_type(s_sta_13, s_end_13, pir_val_13, pir_tim_13, pir_ids_13, timedelta(seconds=seconds))

intervals_2 = detect_pir_activity_type(s_sta_2, s_end_2, pir_val_2, pir_tim_2, pir_ids_2, timedelta(seconds=seconds))
intervals_3 = detect_pir_activity_type(s_sta_3, s_end_3, pir_val_3, pir_tim_3, pir_ids_3, timedelta(seconds=seconds))
intervals_4 = detect_pir_activity_type(s_sta_4, s_end_4, pir_val_4, pir_tim_4, pir_ids_4, timedelta(seconds=seconds))

print(get_activity_vector(intervals_10))
print(get_activity_vector(intervals_2))
print(get_activity_vector(intervals_3))
print(get_activity_vector(intervals_4))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2]
